<a href="https://colab.research.google.com/github/MrA-png/sign-language-video-splitter/blob/main/link_to_gesture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import cv2
import shutil
from yt_dlp import YoutubeDL
from google.colab import files

# 1. Unduh video dari YouTube menggunakan yt-dlp
def download_youtube_video(url, output_path="downloads"):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    ydl_opts = {
        'format': 'mp4',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            video_path = os.path.join(output_path, f"{info_dict['title']}.mp4")
            return video_path
    except Exception as e:
        raise Exception(f"Kesalahan saat mengunduh video: {e}")

# 2. Potong video berdasarkan perubahan gerakan
def segment_video_by_motion(video_path, output_dir="segments1", min_motion_frames=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    prev_frame = None
    segment_count = 0
    frame_buffer = []
    motion_threshold = 100000  # Sensitivitas perubahan gerakan
    active_motion_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

        if prev_frame is None:
            prev_frame = gray_frame
            continue

        frame_diff = cv2.absdiff(prev_frame, gray_frame)
        motion_score = cv2.sumElems(frame_diff)[0]

        if motion_score > motion_threshold:
            frame_buffer.append(frame)
            active_motion_frames += 1
        else:
            if active_motion_frames >= min_motion_frames:
                save_segment(frame_buffer, segment_count, fps, frame_width, frame_height, output_dir)
                segment_count += 1
            frame_buffer = []
            active_motion_frames = 0

        prev_frame = gray_frame

    # Save any remaining frames
    if frame_buffer and active_motion_frames >= min_motion_frames:
        save_segment(frame_buffer, segment_count, fps, frame_width, frame_height, output_dir)

    cap.release()

# 3. Simpan segmen sebagai video
def save_segment(frames, segment_count, fps, frame_width, frame_height, output_dir):
    segment_path = os.path.join(output_dir, f"segment_{segment_count}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(segment_path, fourcc, fps, (frame_width, frame_height))

    for frame in frames:
        out.write(frame)

    out.release()
    print(f"Saved segment: {segment_path}")

# 4. Membuat file ZIP dari folder segments
def create_zip(output_dir="segments1", zip_filename="segments.zip"):
    zip_path = f"{output_dir}.zip"
    shutil.make_archive(output_dir, 'zip', output_dir)
    print(f"ZIP file created: {zip_path}")
    return zip_path

# Main process
def process_video_to_segments(youtube_url):
    print("Downloading video...")
    video_path = download_youtube_video(youtube_url)
    print(f"Video downloaded to {video_path}")

    print("Segmenting video based on motion...")
    segment_video_by_motion(video_path)
    print("Video segmentation completed.")

    print("Creating ZIP file...")
    zip_path = create_zip()
    return zip_path

# URL YouTube
youtube_url = "https://www.youtube.com/watch?v=arAzoJ5aFZ4"

# Proses video dan download ZIP
try:
    zip_path = process_video_to_segments(youtube_url)
    print("Downloading ZIP file...")
    files.download(zip_path)
except Exception as e:
    print(f"Kesalahan: {e}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=arAzoJ5aFZ4
[youtube] arAzoJ5aFZ4: Downloading webpage
[youtube] arAzoJ5aFZ4: Downloading ios player API JSON
[youtube] arAzoJ5aFZ4: Downloading mweb player API JSON
[youtube] arAzoJ5aFZ4: Downloading m3u8 information
[info] arAzoJ5aFZ4: Downloading 1 format(s): 18
[download] Destination: downloads/Kata Dasar dalam Bahasa Isyarat yang perlu kmu ketahui!!!.mp4
[download] 100% of   12.35MiB in 00:00:00 at 14.88MiB/s  
Video downloaded to downloads/Kata Dasar dalam Bahasa Isyarat yang perlu kmu ketahui!!!.mp4
Segmenting video based on motion...
Saved segment: segments1/segment_0.mp4
Saved segment: segments1/segment_1.mp4
Saved segment: segments1/segment_2.mp4
Saved segment: segments1/segment_3.mp4
Saved segment: segments1/segment_4.mp4
Saved segment: segments1/segment_5.mp4
Saved segment: segments1/segment_6.mp4
Saved segment: segments1/segment_7.mp4
Saved segment: segments1/segment_8.mp4
Saved segment: segments1/segment_9.mp4
Save

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>